In [ ]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 22.3 MB 8.1 MB/s 
     |████████████████████████████████| 981 kB 46.6 MB/s 
     |████████████████████████████████| 263 kB 46.9 MB/s 
     |████████████████████████████████| 2.8 MB 31.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 468 kB 33.0 MB/s 
     |████████████████████████████████| 895 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.4 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_excel("/content/drive/MyDrive/Bert/Release_Notes_v0_RawData.xlsx")

In [ ]:
data.head()

,ID,Project_Name,Release_Number,Date,Label,Type,Release_Note
0,1.0,Mozilla Firefox,86.0,2021-02-23,new,new,Firefox now supports simultaneously watching m...
1,2.0,Mozilla Firefox,86.0,2021-02-23,fixed,fixed,Reader mode now works with local HTML pages. U...
2,3.0,Mozilla Firefox,86.0,2021-02-23,changed,changed,"On Linux and Android, the protection to mitiga..."
3,4.0,Mozilla Firefox,86.0,2021-02-23,other,developer,Inactive CSS tool is now showing a warning whe...
4,5.0,Mozilla Firefox,85.0.2,2021-02-09,fixed,fixed,Fixed a deadlock during startup (bug 1679933)


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train=train[["Release_Note","Label"]]
test=test[["Release_Note","Label"]]

In [ ]:
train["Release_Note"]=train["Release_Note"].apply(lambda r: str(r))
train['Label']=train['Label'].astype(str)

In [ ]:
train["Release_Note"].head()

264    Column choice and sort order in "Search Messag...
615    Fixed a bug where log files wouldn't be create...
329    Due to changes in the Mozilla platform profile...
342                               Various security fixes
394    Default to using SSL for XMPP and IRC. This mi...
Name: Release_Note, dtype: object

In [ ]:
test["Release_Note"].head()

696    Added video/audio filtering; you can now acces...
667    Actually fixes the potential game capture cras...
63     Various stability, functionality, and security...
533    Added the ability to pause while recording to ...
66                                 Developer Information
Name: Release_Note, dtype: object

In [ ]:
train["Label"].head()

264         fixed
615       changed
329    unresolved
342         fixed
394       changed
Name: Label, dtype: object

In [ ]:
test["Label"].head()

696    changed
667      fixed
63       fixed
533        new
66       other
Name: Label, dtype: object

In [ ]:
possible_labels = data.Label.unique().tolist()
possible_labels

['new', 'fixed', 'changed', 'other', 'unresolved']

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased' 
t = text.Transformer(MODEL_NAME, maxlen=500, classes=possible_labels)
trn = t.preprocess_train(train["Release_Note"].tolist(), train["Label"].to_numpy())
val = t.preprocess_test(test["Release_Note"].tolist(), test["Label"].to_numpy())
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:422: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 53
	95percentile : 142
	99percentile : 277


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 54
	95percentile : 151
	99percentile : 229


Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

In [ ]:
import time
st_time = time.time()

a=learner.fit_onecycle(5e-5, 6)  #lr=5e-5 , epoch=5

en_time = time.time()

print('Total time: {:.2f}s'.format(en_time-st_time))



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/6
107/107 [==============================] - 191s 2s/step - loss: 1.3190 - accuracy: 0.4750 - val_loss: 1.0014 - val_accuracy: 0.6562
Epoch 2/6
107/107 [==============================] - 163s 2s/step - loss: 0.8136 - accuracy: 0.7281 - val_loss: 0.6940 - val_accuracy: 0.7000
Epoch 3/6
107/107 [==============================] - 163s 2s/step - loss: 0.5187 - accuracy: 0.8141 - val_loss: 0.5139 - val_accuracy: 0.8375
Epoch 4/6
107/107 [==============================] - 163s 2s/step - loss: 0.3273 - accuracy: 0.8891 - val_loss: 0.6982 - val_accuracy: 0.7125
Epoch 5/6
107/107 [==============================] - 163s 2s/step - loss: 0.1402 - accuracy: 0.9563 - val_loss: 0.5373 - val_accuracy: 0.8188
Epoch 6/6
107/107 [==============================] - 163s 2s/step - loss: 0.0599 - accuracy: 0.9875 - val_loss: 0.5399 - val_accuracy: 0.8375
Total time: 1004.86s


In [ ]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

     changed       0.71      0.79      0.75        38
       fixed       0.89      0.86      0.88        66
         new       0.82      0.82      0.82        28
       other       0.92      0.80      0.86        15
  unresolved       0.92      0.92      0.92        13

    accuracy                           0.84       160
   macro avg       0.85      0.84      0.85       160
weighted avg       0.84      0.84      0.84       160



array([[30,  6,  2,  0,  0],
       [ 6, 57,  2,  0,  1],
       [ 4,  0, 23,  1,  0],
       [ 2,  0,  1, 12,  0],
       [ 0,  1,  0,  0, 12]])

In [ ]:
learner.view_top_losses(n=5, preproc=t)

----------
id:138 | loss:7.3 | true:other | pred:changed)

----------
id:123 | loss:6.3 | true:other | pred:changed)

----------
id:150 | loss:5.99 | true:fixed | pred:changed)

----------
id:21 | loss:5.12 | true:fixed | pred:changed)

----------
id:96 | loss:4.55 | true:changed | pred:fixed)



In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
örnek = test.iloc[21]['Release_Note']
print(örnek)

Browser docks now fully shutdown when hidden (prevents playback of any video/audio that may have been playing by browser),Added an option to browser sources, "Reroute audio through OBS", which is enabled by default. This allows the user to turn off the new browser audio handling if they need to for whatever reason (usually only useful when you need to monitor with a different device than what OBS is set to monitor with)


In [ ]:
örnek="Browser docks now fully shutdown when (prevents playback of any video/audio that may have been playing by browser), an option to browser sources, 'Reroute audio through OBS', which is enabled by default. This the user to turn off the new browser audio handling if they need to for whatever reason (usually only useful when you need to monitor with a different device than what OBS is set to monitor with)"

In [ ]:
predictor.predict(örnek)

'changed'

In [ ]:
sample_prediction = predictor.predict(test['Release_Note'].to_list())

In [ ]:
gercek_deger = test["Label"].to_list()

In [ ]:
liste = []
for i in range(160):
  if gercek_deger[i] != sample_prediction[i]:
      print(gercek_deger[i] + " --> " + sample_prediction[i]+ " --> " + str(i))
      liste.append(i)

fixed --> changed --> 1
new --> changed --> 17
new --> changed --> 19
fixed --> changed --> 21
fixed --> unresolved --> 24
changed --> new --> 26
fixed --> changed --> 34
fixed --> new --> 38
changed --> fixed --> 47
fixed --> changed --> 49
unresolved --> fixed --> 50
new --> changed --> 55
changed --> fixed --> 61
changed --> fixed --> 78
other --> new --> 81
new --> changed --> 87
fixed --> changed --> 90
changed --> fixed --> 91
changed --> fixed --> 96
fixed --> new --> 99
other --> changed --> 123
changed --> fixed --> 126
changed --> new --> 131
other --> changed --> 138
new --> other --> 139
fixed --> changed --> 150


In [ ]:
predictor.get_classes()

['changed', 'fixed', 'new', 'other', 'unresolved']

In [ ]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1


In [ ]:
predictor.predict_proba(örnek)

array([9.8717272e-01, 5.9649353e-03, 4.1331006e-03, 6.3448679e-04,
       2.0947522e-03], dtype=float32)

In [ ]:
predictor.explain(örnek)

In [ ]:
target_names = ['new', 'fixed', 'changed', 'other', 'unresolved']

In [ ]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(örnek, predictor.predict_proba)
te.show_prediction(target_names=target_names)

In [ ]:
te.show_prediction(targets=['unresolved'],target_names=target_names)

In [ ]:
te.show_weights(target_names=target_names)

In [ ]:
predictor.save('/content/drive/MyDrive/Bert/bert_model')

In [ ]:
reloaded_predictor = ktrain.load_predictor('/content/drive/MyDrive/Bert/bert_model')

In [ ]:
test_example="On Linux and Android, the protection to mitigate the stack clash attack has been activated. From Firefox 86 onward, DTLS 1.0 is no longer supported for establishing WebRTC's PeerConnections. All WebRTC services need to support DTLS 1.2 from now on as the minimum version. Consolidated all video decoding in the new RDD process which results in a more secure Firefox."

In [ ]:
reloaded_predictor.predict(test_example)

'changed'

In [ ]:
reloaded_predictor.explain(test_example)

In [ ]:
target_names = ['new', 'fixed', 'changed', 'other', 'unresolved']

In [ ]:
!pip install eli5


In [ ]:
import eli5
from eli5.lime import TextExplainer
exp = TextExplainer(random_state=42)
exp.fit(örnek, reloaded_predictor.predict_proba)
exp.show_prediction(target_names=target_names)

In [ ]:
exp.show_weights(target_names=target_names)